In [1]:
from pathlib import Path

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.optim import AdamW
from transformers import WhisperFeatureExtractor, WhisperModel, logging as hf_logging

/usr/lib/python3/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/usr/lib/python3/dist-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/usr/lib/python3/dist-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can

In [2]:
# !pip install librosa

In [3]:

from typing import List, Tuple, Dict, Optional, Union

import torch
from torch.utils.data import Dataset, DataLoader

# Мы используем torchaudio как основной бэкенд (лучше читает opus),
# а librosa — как резервный вариант для подстраховки.
import torchaudio
import numpy as np
import librosa

In [4]:
from tqdm import tqdm

In [5]:
from pathlib import Path
from glob import glob
import json
import random

In [6]:
def _pick_existing(*candidates):
    for p in candidates:
        p = Path(p)
        if (p / "audio").exists():
            return p
    return None

train_dir = _pick_existing("train_data", "train_opus")
test_dir  = _pick_existing("test_data",  "test_opus")

if train_dir is None or test_dir is None:
    raise RuntimeError(
        "Не найдены папки с данными. Ожидались train_data/ или train_opus/ (и аналогично для test_*/)."
    )

print(f"train_dir: {train_dir.resolve()}")
print(f"test_dir : {test_dir.resolve()}")

# Списки файлов
train_files = glob(str(train_dir / "audio" / "*.opus"))
test_files  = glob(str(test_dir  / "audio" / "*.opus"))

print(f"Количество тренировочных файлов: {len(train_files)}")
print(f"Количество тестовых файлов: {len(test_files)}")

# Загрузка разметки
wb_path = train_dir / "word_bounds.json"
if not wb_path.exists():
    raise FileNotFoundError(f"Не найден файл разметки: {wb_path}")

with open(wb_path, "r", encoding="utf-8") as f:
    word_bounds = json.load(f)

print(f"Количество размеченных файлов (по ключам в word_bounds.json): {len(word_bounds)}")

# Базовые счётчики
train_ids = {Path(p).stem for p in train_files}
test_ids  = {Path(p).stem for p in test_files}
pos_ids   = set(word_bounds.keys())

# Положительные — это пересечение ключей разметки с фактически существующими файлами
pos_in_train = pos_ids & train_ids
pos_count    = len(pos_in_train)
neg_count    = len(train_files) - pos_count

print("\nРаспределение классов в train (по наличию id в word_bounds.json):")
print(f"Положительные примеры: {pos_count} ({pos_count / max(len(train_files),1) * 100:.1f}%)")
print(f"Отрицательные примеры: {neg_count} ({neg_count / max(len(train_files),1) * 100:.1f}%)")

# Полезные sanity-check'и
missing_annot_ids = pos_ids - train_ids
if missing_annot_ids:
    print(f"\nПредупреждение: в разметке есть {len(missing_annot_ids)} id, "
          f"для которых не найден файл в {train_dir/'audio'} (первые 5): "
          f"{sorted(list(missing_annot_ids))[:5]}")

test_leak_ids = pos_ids & test_ids
if test_leak_ids:
    print(f"\nПредупреждение: обнаружены {len(test_leak_ids)} id из теста, присутствующие в word_bounds.json "
          f"(первые 5): {sorted(list(test_leak_ids))[:5]}")

# Дубликаты по stem (на всякий случай)
def _dup_stems(paths):
    stems = [Path(p).stem for p in paths]
    seen, dup = set(), set()
    for s in stems:
        if s in seen:
            dup.add(s)
        else:
            seen.add(s)
    return dup

dup_train = _dup_stems(train_files)
dup_test  = _dup_stems(test_files)
if dup_train:
    print(f"\nПредупреждение: дубликаты id в train: {len(dup_train)} (первые 5): {sorted(list(dup_train))[:5]}")
if dup_test:
    print(f"\nПредупреждение: дубликаты id в test: {len(dup_test)} (первые 5): {sorted(list(dup_test))[:5]}")

# Небольшое превью
def _preview(ids_set, n=5):
    ids_list = sorted(list(ids_set))
    random.seed(0)
    return sorted(random.sample(ids_list, min(n, len(ids_list))))

print("\nПримеры id из train:", _preview(train_ids, 5))
print("Примеры id из test :", _preview(test_ids, 5))
print("Примеры POS id     :", _preview(pos_in_train, 5))

train_dir: /home/user/Desktop/a/train_opus
test_dir : /home/user/Desktop/a/test_opus
Количество тренировочных файлов: 90000
Количество тестовых файлов: 27000
Количество размеченных файлов (по ключам в word_bounds.json): 45000

Распределение классов в train (по наличию id в word_bounds.json):
Положительные примеры: 45000 (50.0%)
Отрицательные примеры: 45000 (50.0%)

Примеры id из train: ['0578706700521351846393642918886346925726', '3747623082454920429904507450062128040033', '5600513868334424866262280573577017975423', '6115258489815918371243570488516101842182', '7447369466826958236737655344544781920680']
Примеры id из test : ['0499438862317083278182675188266545621622', '3161809059075456484089887853187931721551', '4695837353874467852979108224819486928786', '5125349328490651777970264902178208387036', '9188837987070080381694995888667050393849']
Примеры POS id     : ['0588658564848284222758254070217825981013', '3743858473378128480264214418181389794712', '5598385971789991196651732353383092448

In [7]:
def load_audio_16k(path: Union[str, Path], sr_target: int = 16000) -> torch.Tensor:
    """
    Загрузка аудио в 16 кГц, моно, float32 -> torch.float32.

    Порядок:
    1) torchaudio.load (часто лучший выбор для .opus при наличии ffmpeg)
    2) librosa.load как резервный путь

    Возвращает:
        wav: Tensor [T] в диапазоне примерно [-1, 1]
    """
    path = str(path)
    try:
        wav, sr = torchaudio.load(path)  # wav: [C, T], float32/float64/…
        # моно
        if wav.dim() == 2:
            if wav.size(0) > 1:
                wav = wav.mean(dim=0, keepdim=True)  # усредняем каналы
            wav = wav.squeeze(0)  # -> [T]

        # ресемплинг при необходимости
        if sr != sr_target:
            wav = torchaudio.functional.resample(wav, orig_freq=sr, new_freq=sr_target)

        # гарантируем float32
        if wav.dtype != torch.float32:
            wav = wav.float()
        return wav
    except Exception:
        # librosa (например, если нет ffmpeg-бэкенда у torchaudio)
        y, sr = librosa.load(path, sr=sr_target, mono=True)
        wav = torch.from_numpy(y.astype(np.float32))
        return wav


def ensure_length(wav: torch.Tensor, target_len: int) -> torch.Tensor:
    """
    Приводим сигнал к заданной длине target_len с помощью паддинга нулями или обрезания.
    Важно для батчинга: у всех сегментов одинаковая длина.
    """
    T = wav.numel()
    if T == target_len:
        return wav
    if T > target_len:
        return wav[:target_len]
    # паддинг справа
    pad = target_len - T
    return torch.nn.functional.pad(wav, (0, pad))


def pick_positive_window(
    T: int,
    sr: int,
    seg_size: int,
    bounds: Tuple[float, float],
    context_frac: float = 0.5,
) -> Tuple[int, int]:
    """
    Возвращает [left, right) индексы окна длины seg_size так, чтобы целевая фраза попала внутрь,
    и при этом оставалось немного контекста до/после (задается долей context_frac).

    Идея:
    - Есть интервал фразы [t0, t1] в секундах. Его длина Lp = (t1 - t0) * sr.
    - Мы хотим поместить этот интервал в окно длины seg_size, желательно не "впритык".
    - Сначала вычисляем допустимый диапазон для левого края окна так, чтобы фраза полностью влезла.
    - Затем случайно сдвигаем окно в этом диапазоне (даёт разнообразие).
    """
    t0, t1 = bounds
    # Границы фразы в сэмплах
    p0 = int(round(t0 * sr))
    p1 = int(round(t1 * sr))
    p0 = max(0, min(T, p0))
    p1 = max(0, min(T, p1))
    pos_len = max(1, p1 - p0)

    # Если фраза длиннее окна, берём центральный фрагмент фразы
    if pos_len >= seg_size:
        center = (p0 + p1) // 2
        left = max(0, center - seg_size // 2)
        right = min(T, left + seg_size)
        left = right - seg_size
        return left, right

    # Иначе фраза короче окна -> разместим её внутри окна с контекстом
    free = seg_size - pos_len

    # Доля контекста до/после. Например, context_frac=0.5 -> симметричный контекст
    # но мы слегка рандомизируем долю слева/справа
    alpha = np.clip(np.random.normal(loc=context_frac, scale=0.15), 0.0, 1.0)
    left_ctx = int(alpha * free)
    right_ctx = free - left_ctx

    left = p0 - left_ctx
    right = p1 + right_ctx

    # Если окно "вылезает" за аудио — подвинем
    if left < 0:
        shift = -left
        left = 0
        right = min(T, right + shift)
    if right > T:
        shift = right - T
        right = T
        left = max(0, left - shift)

    # На всякий случай — фиксированная длина
    if right - left != seg_size:
        right = min(T, left + seg_size)
        left = right - seg_size
        left = max(0, left)
    return left, right


def pick_negative_window(T: int, seg_size: int) -> Tuple[int, int]:
    """
    Простой случай для отрицательного окна: равномерно выбираем позицию,
    если сигнал короче seg_size — окно начнётся с 0, а остаток добьётся нулями.
    """
    if T <= seg_size:
        return 0, min(T, seg_size)
    left = random.randint(0, T - seg_size)
    right = left + seg_size
    return left, right


class KWSDataset(Dataset):
    """
    Базовый датасет для KWS.

    Ожидаемые входы:
        pos_items: список положительных примеров [(path, (start_sec, end_sec)), ...]
        neg_items: список отрицательных примеров [(path, None), ...] или [(path, ()), ...]
    где:
        - path: путь к .opus
        - (start_sec, end_sec): границы фразы из word_bounds.json для данного файла
          (если в файле несколько фрагментов, можно передавать список — см. ниже расширение)

    Параметры:
        seg_size_samples: длина сегмента в сэмплах (для 1 сек при 16кГц: 16000)
        sr: частота дискретизации для загрузки и сегментации (обычно 16000)
        mix_posneg: если True — формируем единый пул и даем DataLoader'у управлять shuffle;
                    если False — можно реализовать собственную балансировку.
        seed: сид для воспроизводимости.
    """

    def __init__(
        self,
        pos_items: List[Tuple[Union[str, Path], Tuple[float, float]]],
        neg_items: List[Tuple[Union[str, Path], Optional[Tuple[float, float]]]],
        seg_size_samples: int,
        sr: int = 16000,
        mix_posneg: bool = True,
        seed: Optional[int] = None,
        allow_negative_inside_positive: bool = False,
    ):
        super().__init__()
        self.sr = sr
        self.seg_size = int(seg_size_samples)
        self.rng = random.Random(seed)

        # Нормализуем вход: приводим пути к Path, фильтруем битые записи
        def _normalize(items, is_pos: bool):
            out = []
            for it in items:
                if isinstance(it, (list, tuple)) and len(it) == 2:
                    p, b = it
                else:
                    # допускаем формат просто (path,) для отрицательных
                    p, b = it, None
                p = Path(p)
                if not p.exists():
                    # лучше явно сигнализировать, но для демонстрационного ноутбука мы молча пропустим
                    continue
                if is_pos and (b is None or len(b) != 2):
                    # у положительного примера обязательно должны быть границы
                    continue
                out.append((p, b))
            return out

        self.pos_items = _normalize(pos_items, is_pos=True)
        self.neg_items = _normalize(neg_items, is_pos=False)

        if mix_posneg:
            # формируем единый пул (path, bounds, label)
            self.samples = [(p, b, 1) for p, b in self.pos_items] + [(p, None, 0) for p, _ in self.neg_items]
        else:
            # можно хранить раздельно и реализовать свою стратегию балансировки в __getitem__
            self.samples = [(p, b, 1) for p, b in self.pos_items] + [(p, None, 0) for p, _ in self.neg_items]

        # Флаг: разрешать ли извлекать отрицательные окна из «пустых зон» положительных файлов
        # (полезно при дефиците отрицательных примеров)
        self.allow_negative_inside_positive = allow_negative_inside_positive

    def __len__(self):
        return len(self.samples)

    def _get_segment(self, wav: torch.Tensor, label: int, bounds: Optional[Tuple[float, float]]) -> torch.Tensor:
        """
        Возвращает сегмент фиксированной длины для данного wav и метки.
        Для label==1 — гарантируем попадание фразы в окно.
        Для label==0 — равномерный выбор окна.

        Если wav слишком короткий — дополняем нулями.
        """
        T = wav.numel()

        # Если аудио пустое (редкий случай), вернем просто нули
        if T == 0:
            return torch.zeros(self.seg_size, dtype=torch.float32)

        if label == 1 and bounds is not None:
            left, right = pick_positive_window(T=T, sr=self.sr, seg_size=self.seg_size, bounds=bounds)
        else:
            left, right = pick_negative_window(T=T, seg_size=self.seg_size)

        seg = wav[left:right]
        seg = ensure_length(seg, self.seg_size)
        return seg

    def __getitem__(self, index: int):
        """
        Возвращает:
            segment: Tensor [seg_size] float32
            label:   int (0/1)
            aux:     словарь с технической информацией (путь, длительность и т.п.) — полезно для отладки
        """
        index = index % len(self.samples)
        path, bounds, label = self.samples[index]

        # Загрузка и нормализация аудио
        wav = load_audio_16k(path, sr_target=self.sr)

        # Нормализация амплитуды по RMS/peak может помочь (опционально).
        # Для простоты — легкий нормировщик по пику с защитой от деления на 0.
        peak = wav.abs().max().item()
        if peak > 0:
            wav = wav / peak

        # Сегмент
        segment = self._get_segment(wav, label, bounds)
        if segment is None:
            print("WARN: segment is None!", path, bounds, label)
            segment = torch.zeros(self.seg_size, dtype=torch.float32)

        aux = {
            "path": str(path),
            "label": label,
            "duration_sec": len(wav) / float(self.sr),
            "bounds": bounds,
        }
        return segment, label, aux


# Сформируем pos/neg списки для датасета.
def build_pos_neg_lists(train_files: List[str], word_bounds: Dict[str, List[float]]) \
        -> Tuple[List[Tuple[str, Tuple[float, float]]], List[Tuple[str, None]]]:
    pos_items, neg_items = [], []
    for f in train_files:
        fid = Path(f).stem
        if fid in word_bounds:
            start, end = word_bounds[fid]
            pos_items.append((f, (float(start), float(end))))
        else:
            neg_items.append((f, None))
    return pos_items, neg_items

In [8]:
# pos_items, neg_items = build_pos_neg_lists(train_files, word_bounds)

# seg_size_samples = 16000  # 1 сек при 16 кГц
# ds = KWSDataset(
#     pos_items=pos_items,
#     neg_items=neg_items,
#     seg_size_samples=seg_size_samples,
#     sr=16000,
#     mix_posneg=True,
#     seed=42,
# )

# def collate_fn(batch):
#     segments, labels, aux = zip(*batch)
#     segments = torch.stack(segments, dim=0)          # [B, T]
#     labels = torch.tensor(labels, dtype=torch.long)  # [B]
#     return segments, labels, aux

# loader = DataLoader(ds, batch_size=32, shuffle=True, num_workers=0, collate_fn=collate_fn)
# xb, yb, auxb = next(iter(loader))
# print(xb.shape, yb.shape, auxb[0])

# говнокод от гпт

In [9]:
class WhisperBinaryClassifier(nn.Module):
    def __init__(self, pretrained_whisper: str = "openai/whisper-small"):
        super().__init__()
        # Загружаем WhisperModel (encoder+decoder). Мы будем использовать encoder outputs.
        self.whisper = WhisperModel.from_pretrained(pretrained_whisper)
        # Размер скрытого слоя энкодера
        enc_hidden = self.whisper.config.d_model  # usually 768/1024/... depending on size
        # Глобальный pooling -> логит
        self.pool = nn.AdaptiveAvgPool1d(1)  # будем применять к [B, C, T] -> [B, C, 1]
        self.classifier = nn.Linear(enc_hidden, 1)

    def forward(self, input_features, attention_mask=None):
        # Используем только encoder
        encoder_outputs = self.whisper.encoder(input_features=input_features, attention_mask=attention_mask)
        pooled = encoder_outputs.last_hidden_state.mean(dim=1)  # [B, hidden]
        logits = self.classifier(pooled).squeeze(-1)  # [B]
        return logits

In [10]:
def prepare_feature_extractor(model_name="openai/whisper-small"):
    fe = WhisperFeatureExtractor.from_pretrained(model_name)
    return fe

In [11]:
def collate_fn_whisper(batch, feature_extractor):
    """
    batch: from your KWSDataset -> (segments [T], label, aux)
    feature_extractor: WhisperFeatureExtractor (transformers)
    Возвращает input_features Tensor [B, seq_len, feat_dim], labels Tensor [B]
    """
    segments, labels, aux = zip(*batch)
    # segments: list of tensors [T]
    # преобразуем в numpy float32 и в список
    segs = [s.numpy() if isinstance(s, torch.Tensor) else s for s in segments]
    # feature_extractor ожидает list[np.ndarray] и вернёт 'input_features' (лог-мел)
    # return_tensors=None -> вернёт списки/np, но мы хотим тензор -> return_tensors='pt'
    feats = feature_extractor(segs, sampling_rate=feature_extractor.sampling_rate, return_tensors="pt").input_features
    labels = torch.tensor(labels, dtype=torch.float32)
    return feats, labels, aux

In [12]:
def compute_metrics_harmonic(y_true: torch.Tensor, y_prob: torch.Tensor, thr: float = 0.5):
    """
    Возвращает FRR, FAR и score = harmonic_mean(1-FRR, 1-FAR)
    y_true: {0,1}
    y_prob: probabilities or logits (будем применять sigmoid)
    """
    with torch.no_grad():
        probs = torch.sigmoid(y_prob) if y_prob.max() > 1.0 or y_prob.min() < 0 else torch.clamp(y_prob, 0.0, 1.0) if y_prob.max() <= 1.0 else torch.sigmoid(y_prob)
        preds = (probs >= thr).long()
        y_true = y_true.long()
        TP = int(((preds == 1) & (y_true == 1)).sum().item())
        FN = int(((preds == 0) & (y_true == 1)).sum().item())
        FP = int(((preds == 1) & (y_true == 0)).sum().item())
        TN = int(((preds == 0) & (y_true == 0)).sum().item())
        NUM_POS = TP + FN
        NUM_NEG = FP + TN
        FRR = FN / NUM_POS if NUM_POS > 0 else 0.0
        FAR = FP / NUM_NEG if NUM_NEG > 0 else 0.0
        one_minus_frr = 1.0 - FRR
        one_minus_far = 1.0 - FAR
        # harmonic mean safe
        if (one_minus_frr + one_minus_far) == 0:
            score = 0.0
        else:
            score = 2 * one_minus_frr * one_minus_far / (one_minus_frr + one_minus_far)
        return {"TP": TP, "FP": FP, "FN": FN, "TN": TN, "FRR": FRR, "FAR": FAR, "score": score}

In [13]:
def train_loop(
    model: nn.Module,
    feature_extractor,
    train_loader: DataLoader,
    val_loader: Optional[DataLoader],
    device: torch.device,
    epochs: int = 5,
    lr: float = 1e-5,
    grad_accum_steps: int = 1,
    save_path: str = "./whisper_kws_ckpt.pt",
):
    model.to(device)
    optimizer = AdamW(model.parameters(), lr=lr)
    criterion = nn.BCEWithLogitsLoss()
    scaler = torch.cuda.amp.GradScaler(enabled=(device.type == "cuda"))
    best_score = -1.0

    for epoch in range(1, epochs + 1):
        model.train()
        total_loss = 0.0
        n_batches = 0
        optimizer.zero_grad()
        for step, (feats, labels, aux) in tqdm(enumerate(train_loader, start=1), total=len(train_loader)):
            feats = feats.to(device)  # [B, seq_len, feat_dim]
            labels = labels.to(device)
            with torch.cuda.amp.autocast(enabled=(device.type == "cuda")):
                logits = model(feats)  # [B]
                loss = criterion(logits, labels)
                loss = loss / grad_accum_steps
            scaler.scale(loss).backward()
            total_loss += loss.item() * grad_accum_steps
            n_batches += 1

            if step % grad_accum_steps == 0:
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()

        avg_loss = total_loss / n_batches if n_batches > 0 else 0.0
        print(f"Epoch {epoch} train_loss={avg_loss:.6f}")

        # Валидация
        if val_loader is not None:
            model.eval()
            all_logits = []
            all_labels = []
            with torch.no_grad():
                for feats, labels, aux in tqdm(val_loader):
                    feats = feats.to(device)
                    labels = labels.to(device)
                    logits = model(feats)
                    all_logits.append(logits.cpu())
                    all_labels.append(labels.cpu())
            all_logits = torch.cat(all_logits, dim=0)
            all_labels = torch.cat(all_labels, dim=0)
            metrics = compute_metrics_harmonic(all_labels, torch.sigmoid(all_logits))
            print(f"VAL metrics: score={metrics['score']:.4f} FRR={metrics['FRR']:.4f} FAR={metrics['FAR']:.4f} TP={metrics['TP']} FP={metrics['FP']} FN={metrics['FN']} TN={metrics['TN']}")

            if metrics["score"] > best_score:
                best_score = metrics["score"]
                # сохраняем чекпоинт
                torch.save({"epoch": epoch, "model_state": model.state_dict(), "optimizer_state": optimizer.state_dict()}, save_path)
                print(f"Saved best checkpoint with score={best_score:.4f} -> {save_path}")

    return model

In [14]:
TRAIN_AUDIO_DIR = "./train_opus/audio"  # поменяй при необходимости
WORD_BOUNDS_PATH = "./train_opus/word_bounds.json"
PRETRAINED_WHISPER = "openai/whisper-small"  # можно 'small', 'base' и т.д.
BATCH_SIZE = 32
SEG_SECONDS = 1.0
SEG_SAMPLES = int(16000 * SEG_SECONDS)
EPOCHS = 3
LR = 3e-5
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [15]:
with open(WORD_BOUNDS_PATH, "r", encoding="utf-8") as f:
    word_bounds = json.load(f)

In [16]:
pos_items, neg_items = build_pos_neg_lists(train_files, word_bounds)

ds = KWSDataset(
    pos_items=pos_items,
    neg_items=neg_items,
    seg_size_samples=SEG_SAMPLES,
    sr=16000,
    mix_posneg=True,
    seed=42,
)

In [17]:
len(ds)

90000

In [ ]:
n = len(ds)
idxs = list(range(n))
random.shuffle(idxs)
n_val = max(1, int(0.1 * n))
val_idxs = set(idxs[:n_val])
train_idx_list = [i for i in idxs if i not in val_idxs]

from torch.utils.data import Subset

ds_train = Subset(ds, train_idx_list)
ds_val = Subset(ds, list(val_idxs))

In [19]:
feature_extractor = prepare_feature_extractor(PRETRAINED_WHISPER)

In [20]:
f = lambda batch: collate_fn_whisper(batch, feature_extractor)


train_loader = DataLoader(ds_train, batch_size=BATCH_SIZE, shuffle=True, collate_fn=f, num_workers=16)
val_loader = DataLoader(ds_val, batch_size=BATCH_SIZE, shuffle=False, collate_fn=f, num_workers=16)

In [21]:
model = WhisperBinaryClassifier(pretrained_whisper=PRETRAINED_WHISPER)

trained_model = train_loop(
    model=model,
    feature_extractor=feature_extractor,
    train_loader=train_loader,
    val_loader=val_loader,
    device=DEVICE,
    epochs=EPOCHS,
    lr=LR,
    grad_accum_steps=1,
    save_path="./whisper_kws_best.pt",
)

/tmp/ipykernel_10953/762828574.py:15: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=(device.type == "cuda"))
  0%|          | 0/2532 [00:00<?, ?it/s]/tmp/ipykernel_10953/762828574.py:26: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(device.type == "cuda")):
100%|██████████| 2532/2532 [40:58<00:00,  1.03it/s]


Epoch 1 train_loss=0.059277


100%|██████████| 282/282 [04:34<00:00,  1.03it/s]


VAL metrics: score=0.9883 FRR=0.0096 FAR=0.0138 TP=4455 FP=62 FN=43 TN=4440
Saved best checkpoint with score=0.9883 -> ./whisper_kws_best.pt


100%|██████████| 2532/2532 [40:58<00:00,  1.03it/s]


...

VAL metrics: score=0.9816 FRR=0.0225 FAR=0.0142 TP=4397 FP=64 FN=101 TN=4438



100%|██████████| 2532/2532 [40:58<00:00,  1.03it/s]


Epoch 3 train_loss=0.034025


100%|██████████| 282/282 [04:34<00:00,  1.03it/s]

VAL metrics: score=0.9880 FRR=0.0091 FAR=0.0149 TP=4457 FP=67 FN=41 TN=4435


# Inference

In [22]:
import os

In [23]:
TEST_PATH = "./test_opus/audio"
files = [f for f in os.listdir(TEST_PATH) if f.endswith(".opus") and not f.startswith("._")]

from torch.utils.data import Dataset, DataLoader

class AudioDataset(Dataset):
    def __init__(self, path, sr=16000):
        self.files = [f for f in os.listdir(path) if f.endswith(".opus") and not f.startswith("._")]
        self.path = path
        self.sr = sr

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        fname = self.files[idx]
        file_id = os.path.splitext(fname)[0]
        waveform, sr = torchaudio.load(os.path.join(self.path, fname))
        waveform = waveform.mean(dim=0)
        if sr != self.sr:
            waveform = torchaudio.functional.resample(waveform, sr, self.sr)
        return file_id, waveform

def collate_fn(batch):
    ids, waves = zip(*batch)
    feats = feature_extractor([w.numpy() for w in waves], sampling_rate=16_000, return_tensors="pt").input_features
    return ids, feats

In [24]:
dataset = AudioDataset(TEST_PATH)
loader = DataLoader(dataset, batch_size=16, collate_fn=collate_fn, num_workers=16)

In [25]:
import pandas as pd

In [30]:
results = []
all_logits = []
model.eval()

with torch.no_grad():
    for ids, feats in tqdm(loader):
        feats = feats.to(DEVICE)
        batch_logits = model(feats).cpu().numpy().ravel()
        all_logits.extend(batch_logits)
        results.extend(ids)

100%|██████████| 1688/1688 [14:11<00:00,  1.98it/s]


In [37]:
df = pd.DataFrame({
    "id": results,
    "logit": all_logits
})

# Теперь можно менять порог сколько угодно раз:
threshold = 0.07
df["label"] = (torch.sigmoid(torch.tensor(df["logit"])) > threshold).int().numpy()
df["label"].value_counts()

1    13554
0    13446
Name: label, dtype: int64

In [38]:
# Превращаем в DataFrame с логитами


df[["id", "label"]].to_csv("sub4.csv", index=False)